# Marmalade QC

Data obtained via hard-disk shipped by Mike, and tarballs extracted into original_data directory. Write permissions have been removed form contents of this directory.

## Validate files

md5sums are provided in the manifest.csv files,but not formatted so that md5sum can use them directly, so a quick reshuffle is needed...


In [2]:
import glob
import subprocess
import sys

In [3]:
manifests=glob.glob("original_data/*manifest.csv")
checkfile=open('md5sums.txt','w')
for manifest in manifests:
    f=open(manifest,'r')
    for line in f:
        if not 'file_path' in line:
            fields=line.split("\t")
            file=fields[0]
            md5sum=fields[1]
            checkfile.write("%s *original_data/%s\n" % (md5sum,file))
    f.close()
checkfile.close()

result=subprocess.run(['md5sum','-c','md5sums.txt'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
sys.stdout.write(result.stdout.decode('utf-8'))
sys.stderr.write(result.stderr.decode('utf-8'))

original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109742/Mock2_S1_L004_R1_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109742/Mock2_S1_L004_R2_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109743/Mock3_S1_L005_R1_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109743/Mock3_S1_L005_R2_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109735/1_S1_L001_R1_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109735/1_S1_L001_R2_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109736/2_S2_L001_R1_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109736/2_S2_L001_R2_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109737/3_S3_L001_R1_001.fastq.gz: OK
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109737/3_S3_L001_R2_001.fastq.g

## QC analysis

The dag-wf-qc-pe workflow will do a QC process for us...First need to co-erce the files into having sensible names.

From Mikes' email:

* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_1_16_2019-04-10.tar - Three poos
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_2_16_2019-04-10.tar - Three lungs
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_3_16_2019-04-10.tar - Mock1
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_4_16_2019-04-10.tar - Mock2
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_5_16_2019-04-10.tar - Mock3

The sample sheets show the samples as follows:

* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_1_16_2019-04-10_SampleSheet.csv: IGF109735, IGF109736, IGF109737
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_2_16_2019-04-10_SampleSheet.csv: IGF109738, IGF109739, IGF109740
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_3_16_2019-04-10_SampleSheet.csv: IGF109741
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_4_16_2019-04-10_SampleSheet.csv: IGF109742
* IGFQ000535_hoyles_12-6-2018_wgs-metagenomics_5_16_2019-04-10_SampleSheet.csv: IGF109743

A mapping file for renaming to was manually created:


In [4]:
%%bash 
cat mapping.txt

original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109735/1_S1_L001_R1_001.fastq.gz	gut1	1
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109735/1_S1_L001_R2_001.fastq.gz	gut1	2
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109736/2_S2_L001_R1_001.fastq.gz	gut2	1
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109736/2_S2_L001_R2_001.fastq.gz	gut2	2
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109737/3_S3_L001_R1_001.fastq.gz	gut3	1
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109737/3_S3_L001_R2_001.fastq.gz	gut3	2
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109738/10BHS00459_S1_L002_R1_001.fastq.gz	lung1	1
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109738/10BHS00459_S1_L002_R2_001.fastq.gz	lung1	2
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metagenomics/IGF109739/10BHS00654_S2_L002_R1_001.fastq.gz	lung2	1
original_data/IGFQ000535_hoyles_12-6-2018_wgs-metage

In [ ]:
%%bash
printf "y\n"|dag-wf-rename-fastq -i . -o new_fastq -m mapping.txt

In [8]:
%%bash

dag-wf-setup -w qc-pe -i fastq -d QC -n Marmalade_QC

Creating dag-wf-qc-pe conda environment
Solving environment: ...working... done

libcblas-3.8.0       | 6 KB      | ########## | 100% 
libblas-3.8.0        | 6 KB      | ########## | 100% 
spectra-0.0.11       | 19 KB     | ########## | 100% 
xopen-0.6.0          | 11 KB     | ########## | 100% 
liblapack-3.8.0      | 6 KB      | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
#
# To activate this environment, use
#
#     $ conda activate dag-wf-qc-pe
#
# To deactivate an active environment, use
#
#     $ conda deactivate


Setting up qc-pe workflow in QC...


Populating QC/fastq directory with data files...

	/homes/jabbott/Projects/Marmalade/fastq/gut1_1.fq.gz -> QC/fastq/gut1_1.fq.gz
	/homes/jabbott/Projects/Marmalade/fastq/gut1_2.fq.gz -> QC/fastq/gut1_2.fq.gz
	/homes/jabbott/Projects/Marmalade/fastq/gut2_1.fq.gz -> QC/fastq/gut2_1.fq.gz
	/homes/jabbott/Projects/Marmalade/fastq/gut2_2.f

The Snakefile was updated to increase the minimum read length to retain in trimming:

```
# Unpaired read length cutoff for retention of read 1 (--length_1)
tg_params['R1_retain']=70
# Unpaired read length cutoff for retention of read 2 (--length_2)
tg_params['R2_retain']=70
```

In [ ]:
%%bash
cd QC
dag-wf-run

![QC/dag.svg](QC/dag.svg)